In [1]:
import warnings
warnings.filterwarnings("ignore")
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from langdetect import detect
from datetime import datetime, timedelta

In [2]:
threads = []
for i in range(8):
    if (i == 0):
        url = "https://forum.warmane.com/forumdisplay.php?f=20"
    else:
        url = f"https://forum.warmane.com/forumdisplay.php?f=20&page={i+1}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        blog = soup.find_all("div", class_="author")
        for b in blog:
            a_tag = b.find('a')
            if a_tag:
                threads.append(a_tag.get('href'))
    else:
        # Print an error message if the request was not successful
        print(f"Error: Unable to fetch the webpage. Status code: {response.status_code}")

In [3]:
threads = [s.split("&")[0] for s in threads]
threads

['showthread.php?t=461776',
 'showthread.php?t=447248',
 'showthread.php?t=461042',
 'showthread.php?t=454893',
 'showthread.php?t=455421',
 'showthread.php?t=452837',
 'showthread.php?t=445626',
 'showthread.php?t=450632',
 'showthread.php?t=450999',
 'showthread.php?t=447848',
 'showthread.php?t=440585',
 'showthread.php?t=447657',
 'showthread.php?t=423611',
 'showthread.php?t=441511',
 'showthread.php?t=442203',
 'showthread.php?t=440722',
 'showthread.php?t=427259',
 'showthread.php?t=437896',
 'showthread.php?t=415161',
 'showthread.php?t=395942',
 'showthread.php?t=435310',
 'showthread.php?t=432839',
 'showthread.php?t=432838',
 'showthread.php?t=431660',
 'showthread.php?t=432233',
 'showthread.php?t=428332',
 'showthread.php?t=428616',
 'showthread.php?t=428559',
 'showthread.php?t=421765',
 'showthread.php?t=417128',
 'showthread.php?t=414333',
 'showthread.php?t=411881',
 'showthread.php?t=411367',
 'showthread.php?t=409706',
 'showthread.php?t=407587',
 'showthread.php?t=4

In [4]:
def is_greeklish(text):
    # Creating a regular expression pattern to match Latin characters (A-Z, a-z)
    greeklish_pattern = re.compile(r'[A-Za-z]+', re.UNICODE)
    # Searching for the pattern in the text
    match = greeklish_pattern.search(text)
    # If a match is found, return True (indicating Greeklish is present)
    return match is not None

In [5]:
column_names = ['Post', 'Language', 'Author','Date','reply-to']
posts_df = pd.DataFrame(columns=column_names)

In [6]:
for link in threads:
    response = requests.get(f"https://forum.warmane.com/{link}")
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
    ol_element = soup.find('ol')
    for li in ol_element.find_all('li'):
        name_class = li.find("div",class_="popupmenu memberaction")
        if (name_class):
            author = name_class.find('strong')
            post = li.find("blockquote", class_="postcontent restore")
            reply_to = link.split("=")[1]
            if is_greeklish(post.text.strip()):
                language = "Greek/Greeklish"
            else:
                language = "English"
            date = li.find("span", class_="date")
            if (date.text.split(" ")[1] == "Days"):
                date = date.text.split(" ")[0]
                current_datetime = datetime.now()
                date_n_days_ago = current_datetime - timedelta(days=int(date))
                formatted_date = date_n_days_ago.strftime("%B %d, %Y")
            elif (date.text.split(" ")[1] == "Weeks"):
                date = date.text.split(" ")[0]
                current_datetime = datetime.now()
                date_n_days_ago = current_datetime - timedelta(weeks=int(date))
                formatted_date = date_n_days_ago.strftime("%B %d, %Y")
            else:
                formatted_date = datetime.strptime(date.text.strip(), "%B %d, %Y")
            new_row = {'Post': post.text.strip(), 'Language': language, 'Author': author.text,'Date': formatted_date,'reply-to' : reply_to}
            posts_df = posts_df.append(new_row, ignore_index=True)        

In [7]:
posts_df

,Post,Language,Author,Date,reply-to
0,"????????? ??????, ??????? ?????? group ? guild...",Greek/Greeklish,anna3,"November 10, 2023",461776
1,Yparxei kapoia guild pou tha xekinisei ston ON...,Greek/Greeklish,anna3,"November 10, 2023",461776
2,egw 8a bw alliance molis 3ekinisei,Greek/Greeklish,SilentHll,"November 11, 2023",461776
3,LORDAERON x1 / Horde\n\r\nΓεια χαρα!\n\r\nΨαχν...,Greek/Greeklish,pipernan,2022-05-19 00:00:00,447248
4,geia sou eimai ston lordaeron. character: Pat...,Greek/Greeklish,LockandRoll18,"October 16, 2023",447248
...,...,...,...,...,...
408,eksartatai apto client pou exeis :) neltharion...,Greek/Greeklish,Huskymudkipz,2015-04-26 00:00:00,278731
409,Παίδες αντιμετωπίζω ένα θέμα. ξεκίνησα να κατε...,Greek/Greeklish,giannhslad111,2015-04-17 00:00:00,294152
410,χμμμ απενεργοποιήσε το Ani-virus ή τουλάχιστο...,Greek/Greeklish,DarkSwan,2015-04-21 00:00:00,294152
411,"Γεια, μήπως κανείς μπορεί να μου γράψει ποια R...",Greek/Greeklish,SpiritNative,2015-04-02 00:00:00,292137


In [9]:
posts_df.to_csv('f3352311.csv', index=False, encoding='utf-8')